
# From Python to Production 
## SQL & Python Integration  
### Beginner → Advanced → Production-Ready  

**By Prerna Joshi** | #25DaysOfDataTech  

> *Where data actually lives — and how Python works with it*



## Why This Notebook?

Real-world data workflows rarely use Python or SQL in isolation.

**The reality:**
- Data lives in databases (SQL)
- Heavy filtering & joins happen in SQL
- Analysis, automation, and ML happen in Python
- Results often go back to the database

This notebook teaches the **handoff** between SQL and Python.



## What You Will Learn

- Connect Python to databases using `sqlite3`
- Run SQL queries from Python
- Load SQL results into pandas
- Decide when to use SQL vs pandas
- Perform joins and aggregations
- Transform data with pandas
- Write results back to SQL
- Handle errors and connections
- Understand SQLAlchemy in production


In [1]:

import sqlite3
import pandas as pd



##  Step 1: Connect to Database


In [2]:

conn = sqlite3.connect("transactions.db")
print("Connected to SQLite database")


Connected to SQLite database



## Step 2: Create Tables


In [3]:

cursor = conn.cursor()

cursor.execute(
    "CREATE TABLE IF NOT EXISTS users ("
    "user_id INTEGER PRIMARY KEY, "
    "city TEXT, "
    "age INTEGER)"
)

cursor.execute(
    "CREATE TABLE IF NOT EXISTS transactions ("
    "transaction_id INTEGER PRIMARY KEY, "
    "user_id INTEGER, "
    "amount REAL, "
    "category TEXT, "
    "status TEXT, "
    "transaction_date TEXT, "
    "FOREIGN KEY(user_id) REFERENCES users(user_id))"
)

conn.commit()
print("Tables created")


Tables created



## Step 3: Insert Data


In [4]:

users_data = [
    (1, "New York", 28),
    (2, "Chicago", 35),
    (3, "San Francisco", 42)
]

transactions_data = [
    (1, 1, 120.5, "Food", "success", "2025-01-01"),
    (2, 1, 300.0, "Travel", "success", "2025-01-05"),
    (3, 2, 45.0, "Grocery", "failed", "2025-01-07"),
    (4, 3, 500.0, "Travel", "success", "2025-01-10")
]

cursor.executemany("INSERT OR IGNORE INTO users VALUES (?, ?, ?)", users_data)
cursor.executemany("INSERT OR IGNORE INTO transactions VALUES (?, ?, ?, ?, ?, ?)", transactions_data)

conn.commit()
print("Sample data inserted")


Sample data inserted



## Step 4: Query Data Using SQL


In [5]:

query = (
    "SELECT category, COUNT(*) AS total_txns, SUM(amount) AS total_amount "
    "FROM transactions WHERE status = 'success' "
    "GROUP BY category"
)
pd.read_sql(query, conn)


,category,total_txns,total_amount
0,Food,1,120.5
1,Travel,2,800.0



## Step 5: Error Handling (Production Thinking)


In [6]:

try:
    bad_query = "SELECT * FROM non_existent_table"
    pd.read_sql(bad_query, conn)
except Exception as e:
    print(f"Query failed: {e}")
    # In production: log, retry, or alert


Query failed: Execution failed on sql 'SELECT * FROM non_existent_table': no such table: non_existent_table



## Step 6: Load into pandas


In [7]:

df = pd.read_sql("SELECT * FROM transactions", conn)
df.head()


,transaction_id,user_id,amount,category,status,transaction_date
0,1,1,120.5,Food,success,2025-01-01
1,2,1,300.0,Travel,success,2025-01-05
2,3,2,45.0,Grocery,failed,2025-01-07
3,4,3,500.0,Travel,success,2025-01-10



## Step 7: SQL vs Pandas Decision Guide

**Use SQL when:**
- Filtering millions of rows
- Joining multiple large tables
- Aggregating data at the database level
- Minimizing data transfer over the wire

**Use pandas when:**
- Complex transformations (apply, custom functions)
- Statistical analysis (correlation, distributions)
- Visualization
- ML feature engineering
- Working with data already in memory

**Example**  
Don't: Load 10M rows into pandas, then filter  
Do: Filter in SQL, load 10K rows into pandas



## Step 8: JOIN Example (SQL's Superpower)

### Interview Tip  
> "I use SQL for joins because databases are optimized for this. Joining in pandas after loading data is inefficient and memory‑intensive."


In [8]:

join_query = (
    "SELECT u.city, u.age, t.category, SUM(t.amount) AS total_spent "
    "FROM users u JOIN transactions t ON u.user_id = t.user_id "
    "WHERE t.status = 'success' "
    "GROUP BY u.city, u.age, t.category "
    "ORDER BY total_spent DESC"
)
pd.read_sql(join_query, conn)


,city,age,category,total_spent
0,San Francisco,42,Travel,500.0
1,New York,28,Travel,300.0
2,New York,28,Food,120.5



##  Step 9: Performance Comparison


In [9]:

# Inefficient: Load all data, then filter in pandas
df_all = pd.read_sql("SELECT * FROM transactions", conn)
filtered_pandas = df_all[df_all["status"] == "success"]

# Efficient: Filter in SQL first
filtered_sql = pd.read_sql(
    "SELECT * FROM transactions WHERE status = 'success'",
    conn
)

print("Rows (pandas filter vs SQL filter):", len(filtered_pandas), len(filtered_sql))


Rows (pandas filter vs SQL filter): 3 3



##  Step 10: Transform Data in pandas


In [10]:

df = pd.read_sql("SELECT * FROM transactions", conn)

# Add calculated columns
df["amount_usd"] = df["amount"] * 1.0
df["is_large_transaction"] = df["amount"] > 200

# Analyze
category_avg = df.groupby("category")["amount_usd"].mean()
category_avg


category
Food       120.5
Grocery     45.0
Travel     400.0
Name: amount_usd, dtype: float64


## Step 11: Write Results Back to SQL


In [11]:

summary_df = df.groupby("category", as_index=False)["amount"].sum()
summary_df.to_sql("category_summary", conn, if_exists="replace", index=False)
pd.read_sql("SELECT * FROM category_summary", conn)


,category,amount
0,Food,120.5
1,Grocery,45.0
2,Travel,800.0



## Step 12: SQLAlchemy Introduction

If not installed, run in a notebook cell:  
`%pip install sqlalchemy`


In [12]:

from sqlalchemy import create_engine

engine = create_engine("sqlite:///transactions.db")
pd.read_sql("SELECT * FROM transactions", engine).head()


,transaction_id,user_id,amount,category,status,transaction_date
0,1,1,120.5,Food,success,2025-01-01
1,2,1,300.0,Travel,success,2025-01-05
2,3,2,45.0,Grocery,failed,2025-01-07
3,4,3,500.0,Travel,success,2025-01-10



## Step 13: Connection Management Best Practice


In [13]:

# Context manager with sqlite3
with sqlite3.connect("transactions.db") as conn2:
    df_count = pd.read_sql("SELECT COUNT(*) AS rows FROM transactions", conn2)
    print(df_count)

# Context manager with SQLAlchemy
from sqlalchemy import create_engine
engine = create_engine("sqlite:///transactions.db")
with engine.connect() as conn3:
    df_head = pd.read_sql("SELECT * FROM transactions", conn3).head()
df_head


   rows
0     4


,transaction_id,user_id,amount,category,status,transaction_date
0,1,1,120.5,Food,success,2025-01-01
1,2,1,300.0,Travel,success,2025-01-05
2,3,2,45.0,Grocery,failed,2025-01-07
3,4,3,500.0,Travel,success,2025-01-10



##  Step 14: Complete Real-World Workflow

###  Interview One-Liner  
> "I use SQL for data access and aggregation, Python for transformation, analysis, and automation."


In [14]:

def daily_sales_report(date):
    # Production-style workflow: SQL -> Python -> SQL
    with sqlite3.connect("transactions.db") as connx:
        query = (
            "SELECT category, SUM(amount) AS revenue "
            "FROM transactions "
            "WHERE status = 'success' AND transaction_date = ? "
            "GROUP BY category"
        )
        df = pd.read_sql(query, connx, params=(date,))
        df["report_date"] = date
        df["rank"] = df["revenue"].rank(ascending=False)
        df.to_sql("daily_reports", connx, if_exists="append", index=False)
        return df

daily_sales_report("2025-01-10")


,category,revenue,report_date,rank
0,Travel,500.0,2025-01-10,1.0



## Step 15: Common Mistakes to Avoid


In [15]:

# Mistake 1: Loading entire tables
# Bad:
# df = pd.read_sql("SELECT * FROM huge_table", conn)
# df_filtered = df[df['date'] > '2025-01-01']

# Good:
# df = pd.read_sql(
#     "SELECT * FROM huge_table WHERE date > '2025-01-01'", 
#     conn
# )

# Mistake 2: SQL injection risk
user_input = "admin' OR '1'='1"
# Bad (do NOT do this):
# unsafe = f"SELECT * FROM users WHERE name = '{user_input}'"

# Good:
safe_query = "SELECT * FROM users WHERE city = ?"
pd.read_sql(safe_query, conn, params=("Chicago",))

# Mistake 3: Not closing connections
# Bad:
# conn = sqlite3.connect("db.db")
# ... connection stays open

# Good:
with sqlite3.connect("transactions.db") as c:
    _ = pd.read_sql("SELECT 1", c)



## The Complete Data Flow

**Extract → Transform → Load Pattern:**

```
┌─────────────────┐
│   Database      │
│     (SQL)       │
└────────┬────────┘
         │ pd.read_sql()
         ↓
┌─────────────────┐
│ pandas DataFrame│
│   (in memory)   │
└────────┬────────┘
         │ transform / analyze
         ↓
┌─────────────────┐
│ pandas DataFrame│
│   (processed)   │
└────────┬────────┘
         │ df.to_sql()
         ↓
┌─────────────────┐
│   Database      │
│  (SQL results)  │
└─────────────────┘
```



## Quick Reference

| Task | Use SQL | Use pandas |
|------|---------|------------|
| Filter 10M+ rows | ✅ | ❌ |
| Join 3+ tables | ✅ | ❌ |
| Aggregate data | ✅ | ✅ |
| Apply custom functions | ❌ | ✅ |
| Statistical analysis | ❌ | ✅ |
| Create visualizations | ❌ | ✅ |
| ML feature engineering | ❌ | ✅ |



## Key Takeaways

- SQL and Python are complementary
- Push heavy work to SQL
- Use pandas for flexibility
- Manage connections carefully
- Think end-to-end



## Take It Further

**Ready to level up?**

Try these challenges:
1. Connect to PostgreSQL or MySQL instead of SQLite
2. Build a multi-table ETL pipeline
3. Add data validation before inserting
4. Schedule this workflow with cron or Airflow
5. Create a REST API that queries your database

**Learn more:**
- SQLAlchemy Documentation: https://docs.sqlalchemy.org/
- pandas SQL Guide: https://pandas.pydata.org/docs/user_guide/io.html#sql-queries
- SQL Style Guide: https://www.sqlstyle.guide/
